[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter3_seminar_notebook.ipynb)

---

# Chapter 3 Seminar: ARIMA Models - Practice Exercises

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Seminar Objectives

1. Practice unit root testing with real data
2. Apply the Box-Jenkins methodology
3. Fit and diagnose ARIMA models
4. Generate and evaluate forecasts
5. Work with financial and macroeconomic time series

## Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Time series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
import yfinance as yf

# Auto-ARIMA
try:
    import pmdarima as pm
except:
    !pip install pmdarima -q
    import pmdarima as pm

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22'}

print("Setup complete!")

## Exercise 1: Unit Root Testing

### Task
Download stock price data and determine if it's stationary. Test both prices and returns.

In [ ]:
# Download Apple stock data
ticker = "AAPL"
data = yf.download(ticker, start='2020-01-01', end='2024-12-31', progress=False)

# Handle MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

prices = data['Close']
returns = prices.pct_change().dropna() * 100  # Percentage returns

print(f"Downloaded {ticker} data: {len(prices)} observations")
print(f"Period: {prices.index[0].date()} to {prices.index[-1].date()}")

In [ ]:
# Plot prices and returns
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(prices.index, prices.values, color=COLORS['blue'], linewidth=1, label='Price')
axes[0].set_title(f'{ticker} Stock Price', fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), frameon=False)

axes[1].plot(returns.index, returns.values, color=COLORS['green'], linewidth=0.5, label='Returns')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title(f'{ticker} Daily Returns (%)', fontweight='bold')
axes[1].set_ylabel('Return (%)')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
def comprehensive_unit_root_test(series, name):
    """Run ADF and KPSS tests with interpretation."""
    print(f"\n{'='*60}")
    print(f"Unit Root Tests: {name}")
    print('='*60)
    
    # ADF Test
    adf = adfuller(series.dropna(), autolag='AIC')
    print(f"\nADF Test (H0: Unit root exists)")
    print(f"  Statistic: {adf[0]:.4f}")
    print(f"  p-value: {adf[1]:.6f}")
    print(f"  Lags used: {adf[2]}")
    adf_result = "Stationary" if adf[1] < 0.05 else "Non-stationary"
    print(f"  Result: {adf_result}")
    
    # KPSS Test
    kpss_stat = kpss(series.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test (H0: Stationary)")
    print(f"  Statistic: {kpss_stat[0]:.4f}")
    print(f"  p-value: {kpss_stat[1]:.4f}")
    kpss_result = "Stationary" if kpss_stat[1] > 0.05 else "Non-stationary"
    print(f"  Result: {kpss_result}")
    
    # Combined interpretation
    print(f"\nCombined Conclusion:")
    if adf_result == "Stationary" and kpss_result == "Stationary":
        print("  Both tests agree: STATIONARY")
    elif adf_result == "Non-stationary" and kpss_result == "Non-stationary":
        print("  Both tests agree: NON-STATIONARY")
    else:
        print(f"  Conflicting results: ADF={adf_result}, KPSS={kpss_result}")
    
    return adf_result, kpss_result

In [ ]:
# Test prices
comprehensive_unit_root_test(prices, f"{ticker} Prices")

In [ ]:
# Test returns
comprehensive_unit_root_test(returns, f"{ticker} Returns")

### Exercise 1 Questions

1. What is the order of integration for stock prices?
2. Why are returns stationary while prices are not?
3. What economic interpretation does this have?

## Exercise 2: ACF/PACF Analysis

### Task
Analyze ACF and PACF patterns to identify appropriate model orders.

In [ ]:
# ACF/PACF of prices vs returns
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Prices
plot_acf(prices.dropna(), ax=axes[0, 0], lags=30, color=COLORS['blue'])
axes[0, 0].set_title('ACF: Stock Prices (slow decay = non-stationary)', fontweight='bold')

plot_pacf(prices.dropna(), ax=axes[0, 1], lags=30, color=COLORS['blue'], method='ywm')
axes[0, 1].set_title('PACF: Stock Prices', fontweight='bold')

# Returns
plot_acf(returns.dropna(), ax=axes[1, 0], lags=30, color=COLORS['green'])
axes[1, 0].set_title('ACF: Returns (near zero = stationary)', fontweight='bold')

plot_pacf(returns.dropna(), ax=axes[1, 1], lags=30, color=COLORS['green'], method='ywm')
axes[1, 1].set_title('PACF: Returns', fontweight='bold')

plt.tight_layout()
plt.show()

print("Observation: Prices show slow ACF decay (unit root), returns are nearly white noise")

## Exercise 3: Box-Jenkins Methodology

### Task
Apply the complete Box-Jenkins procedure to model a time series.

**Steps:**
1. Plot data and check for trends
2. Test for unit roots
3. Difference if necessary
4. Identify p, q from ACF/PACF
5. Estimate model
6. Diagnose residuals
7. Forecast

In [ ]:
# Generate a more interesting dataset: simulate an ARIMA(1,1,1) process
np.random.seed(42)
n = 400
phi, theta = 0.6, 0.3

# Generate ARMA(1,1) for differences
eps = np.random.randn(n)
diff_y = np.zeros(n)
for t in range(1, n):
    diff_y[t] = phi * diff_y[t-1] + eps[t] + theta * eps[t-1]

# Integrate to get I(1) series
y = 100 + np.cumsum(diff_y)  # Starting at 100

# Split into train/test
train_size = 350
y_train = y[:train_size]
y_test = y[train_size:]

print(f"Training set: {len(y_train)} observations")
print(f"Test set: {len(y_test)} observations")
print(f"True model: ARIMA(1,1,1) with φ={phi}, θ={theta}")

In [ ]:
# Step 1: Plot the data
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(range(len(y_train)), y_train, color=COLORS['blue'], linewidth=1, label='Training')
ax.plot(range(len(y_train), len(y)), y_test, color=COLORS['orange'], linewidth=1, label='Test')
ax.axvline(x=train_size, color='black', linestyle='--', alpha=0.5)
ax.set_title('Step 1: Visual Inspection', fontweight='bold')
ax.set_xlabel('Time')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2, frameon=False)
plt.tight_layout()
plt.show()

print("Observation: Series shows a stochastic trend (no deterministic pattern)")

In [ ]:
# Step 2: Unit root test
comprehensive_unit_root_test(pd.Series(y_train), "Training Data")

In [ ]:
# Step 3: First difference
diff_train = np.diff(y_train)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(diff_train, color=COLORS['green'], linewidth=0.8, label='ΔY_t')
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('Step 3: First Differenced Series', fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

# Test differenced series
adf_diff = adfuller(diff_train)
axes[1].text(0.5, 0.7, f"ADF on ΔY_t:", transform=axes[1].transAxes, fontsize=14, fontweight='bold', ha='center')
axes[1].text(0.5, 0.5, f"Statistic: {adf_diff[0]:.4f}", transform=axes[1].transAxes, fontsize=12, ha='center')
axes[1].text(0.5, 0.35, f"p-value: {adf_diff[1]:.6f}", transform=axes[1].transAxes, fontsize=12, ha='center')
result = "STATIONARY" if adf_diff[1] < 0.05 else "NON-STATIONARY"
axes[1].text(0.5, 0.2, f"Result: {result}", transform=axes[1].transAxes, fontsize=12, fontweight='bold', ha='center',
            color=COLORS['green'] if result=='STATIONARY' else COLORS['red'])
axes[1].axis('off')
axes[1].set_title('Unit Root Test on Differences', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Conclusion: d = 1 (one difference makes the series stationary)")

In [ ]:
# Step 4: ACF/PACF of differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(diff_train, ax=axes[0], lags=25, color=COLORS['blue'])
axes[0].set_title('Step 4a: ACF of ΔY_t', fontweight='bold')

plot_pacf(diff_train, ax=axes[1], lags=25, color=COLORS['blue'], method='ywm')
axes[1].set_title('Step 4b: PACF of ΔY_t', fontweight='bold')

plt.tight_layout()
plt.show()

print("Identification:")
print("- ACF: Decays (suggests MA component or ARMA)")
print("- PACF: Decays (suggests AR component or ARMA)")
print("- Both decay → ARMA structure in differences → ARIMA")

In [ ]:
# Step 5: Model estimation - compare different specifications
print("Step 5: Model Comparison")
print("="*60)
print(f"{'Model':<20} {'AIC':>12} {'BIC':>12}")
print("-"*60)

models_to_try = [
    (0, 1, 0), (1, 1, 0), (0, 1, 1), 
    (1, 1, 1), (2, 1, 0), (0, 1, 2), 
    (2, 1, 1), (1, 1, 2)
]

results_comparison = {}
for order in models_to_try:
    try:
        model = ARIMA(y_train, order=order)
        res = model.fit()
        results_comparison[order] = {'aic': res.aic, 'bic': res.bic, 'model': res}
        print(f"ARIMA{order:<13} {res.aic:>12.2f} {res.bic:>12.2f}")
    except Exception as e:
        pass

# Find best model
best_aic = min(results_comparison.items(), key=lambda x: x[1]['aic'])
best_bic = min(results_comparison.items(), key=lambda x: x[1]['bic'])

print("-"*60)
print(f"Best by AIC: ARIMA{best_aic[0]}")
print(f"Best by BIC: ARIMA{best_bic[0]}")

In [ ]:
# Use auto_arima for comparison
auto_model = pm.auto_arima(
    y_train, 
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    d=None,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    trace=True
)

print(f"\nAuto-ARIMA selected: {auto_model.order}")

In [ ]:
# Fit the best model and show summary
best_order = best_aic[0]
best_model = ARIMA(y_train, order=best_order).fit()

print(f"\nStep 5: Fitting ARIMA{best_order}")
print("="*60)
print(best_model.summary())

In [ ]:
# Step 6: Diagnostic checking
residuals = best_model.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Step 6: Residual Diagnostics', fontsize=14, fontweight='bold')

# Residuals over time
axes[0, 0].plot(residuals, color=COLORS['blue'], linewidth=0.5, label='Residuals')
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')
axes[0, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

# Histogram
axes[0, 1].hist(residuals, bins=30, color=COLORS['blue'], edgecolor='black', 
                alpha=0.7, density=True, label='Residuals')
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, residuals.mean(), residuals.std()), 
                color=COLORS['red'], linewidth=2, label='Normal')
axes[0, 1].set_title('Residual Distribution', fontweight='bold')
axes[0, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

# ACF of residuals
plot_acf(residuals, ax=axes[1, 0], lags=20, color=COLORS['blue'])
axes[1, 0].set_title('ACF of Residuals (should be white noise)', fontweight='bold')

# Q-Q plot
(osm, osr), (slope, intercept, r) = stats.probplot(residuals, dist="norm")
axes[1, 1].scatter(osm, osr, color=COLORS['blue'], s=20, alpha=0.5, label='Sample')
axes[1, 1].plot(osm, slope*osm + intercept, color=COLORS['red'], linewidth=2, label='Theoretical')
axes[1, 1].set_title('Q-Q Plot', fontweight='bold')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
# Ljung-Box test
lb_test = acorr_ljungbox(residuals, lags=[10, 15, 20], return_df=True)
print("Ljung-Box Test:")
print("="*50)
print(lb_test)
print("\nConclusion:", "Residuals are white noise (p > 0.05)" if all(lb_test['lb_pvalue'] > 0.05) else "Model inadequate")

In [ ]:
# Step 7: Forecasting
forecast_steps = len(y_test)
forecast = best_model.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Handle both DataFrame and array formats
if hasattr(forecast_ci, 'iloc'):
    ci_lower = forecast_ci.iloc[:, 0]
    ci_upper = forecast_ci.iloc[:, 1]
else:
    ci_lower = forecast_ci[:, 0]
    ci_upper = forecast_ci[:, 1]

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

# Training data (last 100 points)
ax.plot(range(250, train_size), y_train[250:], color=COLORS['blue'], linewidth=1.5, label='Training')

# Test data (actual)
test_index = range(train_size, len(y))
ax.plot(test_index, y_test, color=COLORS['orange'], linewidth=1.5, label='Actual')

# Forecast
ax.plot(test_index, forecast_mean, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(test_index, ci_lower, ci_upper, color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=train_size, color='black', linestyle='-', alpha=0.3)
ax.set_title(f'Step 7: Forecasting with ARIMA{best_order}', fontweight='bold')
ax.set_xlabel('Time')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4, frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# Forecast accuracy metrics
forecast_array = np.array(forecast_mean)
actual_array = np.array(y_test)

mae = np.mean(np.abs(forecast_array - actual_array))
rmse = np.sqrt(np.mean((forecast_array - actual_array)**2))
mape = np.mean(np.abs((actual_array - forecast_array) / actual_array)) * 100

print("Forecast Accuracy Metrics:")
print("="*40)
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

## Exercise 4: Real Economic Data - Exchange Rates

### Task
Model EUR/USD exchange rate using ARIMA.

In [ ]:
# Download EUR/USD data
eurusd = yf.download('EURUSD=X', start='2020-01-01', end='2024-12-31', progress=False)

if isinstance(eurusd.columns, pd.MultiIndex):
    eurusd.columns = eurusd.columns.droplevel(1)

fx = eurusd['Close'].dropna()
print(f"EUR/USD Data: {len(fx)} observations")

# Plot
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(fx.index, fx.values, color=COLORS['blue'], linewidth=1, label='EUR/USD')
ax.set_title('EUR/USD Exchange Rate', fontweight='bold')
ax.set_ylabel('Exchange Rate')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# Unit root test
comprehensive_unit_root_test(fx, "EUR/USD")

In [ ]:
# Auto ARIMA
fx_values = fx.values
train_end = len(fx_values) - 50

fx_train = fx_values[:train_end]
fx_test = fx_values[train_end:]

fx_auto = pm.auto_arima(
    fx_train,
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    d=None,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    trace=True
)

print(f"\nSelected model: ARIMA{fx_auto.order}")

In [ ]:
# Forecast
fc, conf = fx_auto.predict(n_periods=len(fx_test), return_conf_int=True)

# Plot
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(fx.index[:train_end][-100:], fx_train[-100:], color=COLORS['blue'], linewidth=1.5, label='Training')
ax.plot(fx.index[train_end:], fx_test, color=COLORS['orange'], linewidth=1.5, label='Actual')
ax.plot(fx.index[train_end:], fc, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(fx.index[train_end:], conf[:, 0], conf[:, 1], color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=fx.index[train_end], color='black', linestyle='-', alpha=0.3)
ax.set_title(f'EUR/USD Forecast with ARIMA{fx_auto.order}', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4, frameon=False)
plt.tight_layout()
plt.show()

# Accuracy
rmse = np.sqrt(np.mean((fc - fx_test)**2))
print(f"\nForecast RMSE: {rmse:.6f}")

## Exercise 5: Practice Questions

Answer the following questions based on your analysis:

1. **Why do we test for unit roots before fitting an ARIMA model?**

2. **What happens if we fit an ARMA model to non-stationary data?**

3. **How do you interpret an ADF p-value of 0.15?**

4. **If KPSS rejects but ADF doesn't reject, what does this suggest?**

5. **Why do ARIMA forecast intervals grow over time?**

## Summary

### What We Practiced

1. **Unit Root Testing**: ADF and KPSS tests with real data
2. **Box-Jenkins Methodology**: Complete workflow from identification to forecasting
3. **Model Selection**: Using AIC, BIC, and auto_arima
4. **Diagnostics**: Residual analysis, Ljung-Box test
5. **Forecasting**: Point forecasts and confidence intervals

### Key Takeaways

- Stock prices are typically I(1), returns are I(0)
- Use both ADF and KPSS for robust conclusions
- ARIMA forecasts for I(1) series have growing uncertainty
- Always validate model with residual diagnostics